
# How to build an Automated Quant Database

Quants analyze market data, often using internal databases at firms like Jane Street, Man Group, and Goldman Sachs. Creating your own stock price database is essential for integrating various data points—like stock prices, economic trends, and custom analytics—to enhance market research efficiency.

With the rise of free market data, now is an ideal time to start storing and analyzing it. This issue will guide you on how to:

- Use SQLite to build a database
- Download stock data for free
- Store the data in a database
- Automate the entire process

Sources:
- https://www.sqlite.org/
- https://docs.python.org/3/library/sqlite3.html
- https://www.pyquantnews.com/the-pyquant-newsletter/how-to-build-an-automated-quant-database

In [ ]:
pip install yfinance

In [ ]:
from sys import argv

import pandas as pd
import yfinance as yf
import sqlite3